In [1]:
duration <- function(fluxo, taxa, periodo){
  periodo <- 1:periodo
  duratio <- (sum((fluxo/(1+taxa)^periodo)*periodo)/sum((fluxo/(1+taxa)^periodo)))
  return(duratio)
}
duration_mod <- function(duration, nova_taxa){
    duratio_mod <- -duration/(1+nova_taxa)
    return(duratio_mod)
}

In [2]:
vpl <- function(fluxo, taxa, investimento, periodo = length(fluxo)){
  periodo <- c(1:periodo)
  vpl <- sum(fluxo/((1+taxa)^periodo))-investimento
  return(vpl)
}
tir_mc <- function(fluxo, n_taxas = 10000, investimento, periodo = length(fluxo),
                taxa_min = -0.99999, taxa_max = 1){
  taxas <- seq(taxa_min,taxa_max,l = n_taxas)
  tir1 <- sapply(taxas,vpl, investimento = investimento, 
                 fluxo = fluxo, periodo = length(fluxo))
  taxas[which.min(abs(tir1))]
}
tir <- function(fluxo, investimento){
  tir <- Re(1/polyroot(c(-investimento, fluxo))-1)
  tir1 <- numeric(length(tir))
  for(i in 1:length(tir1)){
    tir1[i] <- round(vpl(fluxo, tir[i], investimento),7)
  }
  tir <- tir[which(tir1 == min(abs(tir1)))]
  tir <- tir[which.max(tir)]
  return(tir)
}

In [3]:
fluxo <- c(rep(0.1,10),1.1)
periodo <- 11
taxa <- 0.1
dur <- duration(fluxo = fluxo, periodo = periodo, taxa = taxa)
dur

[1] 7.144567

In [4]:
dur_mod <- duration_mod(duration = dur, 0.08)
dur_mod

[1] -6.61534

In [13]:
investimento <- 100
fluxo <- 100*cumprod(rnorm(20, mean = 0.9, sd = 0.1))
taxa <- tir(fluxo, investimento)
taxa
round(vpl(fluxo = fluxo,taxa = taxa, investimento = investimento),10)

[1] 0.8756443

[1] 0

In [14]:
taxa_m <- tir_mc(fluxo = fluxo, n_taxas = 100000, investimento = investimento)
taxa_m

[1] 0.8756394